In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy import stats
import numpy as np
import pickle
import pprint # Pretty Printer!
import warnings
import statistics

Use the tag csv. Check the locatin data to see if there are any commets about using only a left or right mic.
Next, get the tag amplitude per species detection.
Filter for the 'transcribed' sites.
Get the mean amp per song
Since this is detection only data, we can use the task data, somehow merge it with the tag so each detection has one amplitude attached to it. Then, as a condition, change the value of detectoin given the amp.
I'll have to get that condition. It depends on the ARU unit (SM2 or not), openness, and species.
The script to do this is in AbundanceConditionalOccupancy, called distance truncation.ipynb.

# Prepare the data for truncation

In [19]:
raw_Path = r"C:\Users\ilebe\Documents\!Masters!\RETN Community\0_Data\Raw\Wildtrax Limited Percept"
processed_Path = r"C:\Users\ilebe\Documents\!Masters!\RETN Community\0_Data\Processed"
# Set your working directory
site_detections = pd.read_csv(os.path.join(processed_Path, "Limited perceptibility", 'Multi_spp_occupancy.csv'))
site_detections.drop(labels=['Unnamed: 0', 'species_code'], axis=1, inplace=True)
site_detections.head()

,location,recording_date_time,latitude,longitude,WTSP,TEWA,AMCR,LEYE,WISN,MAWA,...,BRCR,GCKI,GRYE,FOSP,DOWO,SOSA,DUFL,TOSO,COLO,PAWR
0,H-CC-1-101,2022-06-06 07:51:00,52.621107,-115.168551,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,H-CC-1-104,2022-06-07 07:50:00,52.618950,-115.155674,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,H-CC-1-108N,2022-06-24 07:56:00,54.415680,-116.361990,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,H-CC-1-115N,2022-06-22 08:18:00,53.430128,-117.467584,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,H-CC-1-122,2022-06-04 07:52:00,52.761806,-116.098022,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
site_detections.to_csv(os.path.join(processed_Path, "Limited perceptibility", "detection_locations.csv"), index=False)

Make a dataframe with the same format as this, but with the amplitudes instead. Can match with the combination of the column and location, from the tags raw data

In [14]:
tags = pd.read_csv(os.path.join(raw_Path, "BU_Harvest_Patch_Retention_-_Community_&_Limited_perceptibility_tag_report.csv"))
tags.head(3)
tags.columns
tags = tags[['location', 'recording_date_time', 'species_code', 'left_freq_filter_tag_peak_level_dbfs', 'right_freq_filter_tag_peak_level_dbfs']]

In [18]:
tags['mean_amp'] = tags[['left_freq_filter_tag_peak_level_dbfs', 'right_freq_filter_tag_peak_level_dbfs']].mean(axis=1)
tags.head(2)

,location,recording_date_time,species_code,left_freq_filter_tag_peak_level_dbfs,right_freq_filter_tag_peak_level_dbfs,mean_amp
0,H22-CC-0,2022-06-18 06:51:00,LEYE,-33.68,-35.28,-34.480
1,H22-CC-0,2022-06-18 06:51:00,MAWA,-23.76,-18.15,-20.955


In [23]:
tags.dropna(inplace=True)

In [21]:
tags[['location', 'recording_date_time', 'species_code', 'mean_amp']].to_csv(os.path.join(processed_Path, "Limited perceptibility", "species_detection_amplitude.csv"), index=False)

Create a dataframe of the amplitude of each *loudest* detection

In [26]:
# Pivot species_amplitude_df to wide format where each species becomes a column
species_amp_pivot = tags.pivot_table(
    index=["location", "recording_date_time"], 
    columns="species_code", 
    values="mean_amp",
    aggfunc='first'  # Assuming there's only one entry per species per location and date
).reset_index()

species_amp_pivot.fillna(value=0, inplace=True)
species_amp_pivot.head(3)

species_code,location,recording_date_time,ALFL,AMCR,AMRE,AMRO,ATTW,BAWW,BBMA,BCCH,...,WETA,WEWP,WISN,WIWA,WIWR,WTSP,WWCR,YBFL,YEWA,YRWA
0,H-CC-1-101,2022-06-06 07:51:00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,-49.37,0.0,0.0,0.000,-38.145,0.0,0.0,0.00
1,H-CC-1-102,2022-06-03 05:53:00,-44.68,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-48.92,0.0,0.00,0.0,0.0,-30.610,0.000,0.0,0.0,0.00
2,H-CC-1-108N,2022-06-24 07:56:00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.0,-44.315,0.000,0.0,0.0,-40.67


# Prepare truncated dataframes

In [27]:
# load predictive dataframe from the playback model
covariates_path = "C:/Users/ilebe/Documents/!Masters!/Analysis/0. Data/Processed/covariates"
predicted_amps = pd.read_csv(r"C:\Users\ilebe\Documents\!Masters!\Playback analysis\0. Data\Processed\real data\predicted_distance_amplitudes_500_sm2.csv")
SM2_status = pd.read_csv(os.path.join(covariates_path, "SM2 status.csv"))

Add whether each recording was taken with an SM2 or not

In [28]:
data_to_truncate_sm2 = pd.merge(species_amp_pivot, SM2_status, on='location')

In [30]:
data_to_truncate_sm2.isna().any()

location               False
recording_date_time    False
ALFL                   False
AMCR                   False
AMRE                   False
                       ...  
WWCR                   False
YBFL                   False
YEWA                   False
YRWA                   False
SM2                    False
Length: 70, dtype: bool

Map the reference species to the actual species. Will need to delete the speceis fo rwhich this quesotin doesn't make sense

In [ ]:
# make a function that identifies the amplitude at which to truncate in forest and open for each bird.
spp_dict = {
    "WTSP": "VESP",
    "RCKI": "AMRO",
    "TEWA": "AMRO",
    "OSFL": "WEME",
    "YRWA": "CCSP",
    "REVI": "BRBL"
}


def map_spp(reference_spp):
    for key, value in spp_dict.items():
        if reference_spp == value:
            return key
    return None  # Return None or any default value you prefer if no match is found

# Now, apply this function to create a new column in your DataFrame
predicted_amps['spp'] = predicted_amps['species_code'].apply(map_spp)


In [ ]:
covariates_path = "C:/Users/ilebe/Documents/!Masters!/Analysis/0. Data/Processed/covariates"
harv_age = pd.read_csv(os.path.join(covariates_path, "Harv_age.csv"))
tags_and_harv_age = pd.merge(filtered_transcribed_visits, harv_age, on = 'location')
tags_and_harv_age = tags_and_harv_age[['location', 'recording_date_time', 'species_code', 'Year_since_logging', 'mean_amp']]
tags_and_harv_age.to_csv(os.path.join(results_folder, "ready_to_truncate.csv"), index=False)

For the truncation, change the so that it doesn't get rid of the speices, but instead changes the detectgion to 0.